In [ ]:
from network_definitions import VGG16

In [ ]:
from __future__ import print_function
import torch
from torchvision import transforms
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np

epsilons = [0, .05, .1, .15, .2, .25, .3]

mean_cifar10 = [0.485, 0.456, 0.406]   
std_cifar10 = [0.229, 0.224, 0.225]
batch_size = 100

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10,std_cifar10),
])

testset = torchvision.datasets.CIFAR10(root='../FlexibleCNNs/data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

print("CUDA Available: ",torch.cuda.is_available())
use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")


In [ ]:
def fgsm_attack(image, epsilon, data_grad):

    sign_data_grad = data_grad.sign()     # Collect the element-wise sign of the data gradient
    perturbed_image = image + epsilon*sign_data_grad # adjusting each pixel of the input image
    perturbed_image = torch.clamp(perturbed_image, -2.117, 2.64)     # adding clipping to maintain [0,1] range

    return perturbed_image

def test( model, device, test_loader, epsilon ):

    correct = 0
    adv_examples = []

    for data, target in test_loader:

        data, target = data.to(device), target.to(device)

        data.requires_grad = True

        output = model(data)
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        loss = F.nll_loss(output, target)

        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data

        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        output = model(perturbed_data)  # Re-classify the perturbed image

        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        for i in range(0,len(final_pred)-1):
            if final_pred[i].sub(target[i]) == 0:
                correct += 1
                # Special case for saving 0 epsilon examples
                if (epsilon == 0) and (len(adv_examples) < 5):
                    adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                    adv_examples.append( (init_pred[i], final_pred[i], adv_ex) )
            else:
                # Save some adv examples for visualization later
                if len(adv_examples) < 5:
                    adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                    adv_examples.append( (init_pred[i], final_pred[i], adv_ex) )

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))

    return final_acc, adv_examples

In [ ]:
#---------------------------------MAIN LOOP TO FIND AVERAGE----------------------------------------------

# Loop to run for 10 iterations then find the average and standard error of the mean:
accuracies = np.zeros((10, len(epsilons))) 

for i in range(10):
    print("-------Iteration:",i,"--------")
    model = VGG16().to(device)
    state = torch.load('./models/model_name/saved_weights.pth')
    model.load_state_dict(state['model'])
    model.eval()

    # Evaluate model
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images.cuda())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.cuda()).sum().item()

    print('Accuracy on the test dataset is: %f %%' % (100 * correct / total))

    # Run test for each epsilon
    for j, eps in enumerate(epsilons):
        print("j:", j,"   eps:", eps)
        acc, ex = test(model, device, testloader, eps)
        print("acc:", acc)
        accuracies[i,j] = acc


In [ ]:
#Save
np.save(f"FGSM_acc_10iterations_model_{model.name}", accuracies)

#Calculate averages:
avg_acc = np.mean(accuracies, axis = 0)

#Calculate error:
sem = np.std(accuracies, axis = 0) / np.sqrt(10)   #standard error over 10 samples

#Save average values:
np.save(f"FGSM_acc_average_{model.name}", avg_acc)
np.save(f"FGSM_acc_sem_{model.name}", sem)

In [ ]:
#Plot average:
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))
plt.plot(epsilons, avg_acc, "*-", label = "Flexible VGG16")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("FGSM Testing Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Testing Accuracy")
plt.legend()
plt.show()


In [ ]:
#Plot average with error bars:
plt.figure(figsize=(5,5))
plt.errorbar(epsilons, avg_acc, yerr= sem)
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("FGSM Testing Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Testing Accuracy")
plt.legend()
plt.show()